In [13]:
import arcpy
import os
import glob
import pandas as pd
import numpy as np
import geopandas as gpd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from shapely import Point
import ipdb

In [43]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import datetime as dt
from datetime import date
import getpass
from shapely.validation import explain_validity
from shapely.validation import make_valid

In [45]:
def get_AGOL_layer(layer_id, crs, output_file=None): 
    """
    This function loads the layer from ArcGIS Online, converts it to pandas geodataframe, projects, 
    and saves to disc as a new file with current date in the name.
    
    Parameters:
    layer_id* (str): ArcGIS Online layer ID can be retrieved from its browser link. For example, in 
    "https://tnc.maps.arcgis.com/home/item.html?id=23fcdb5591ae4889a054b63bcbd7fc98" the ID comes after "id=" and 
    is equal to 23fcdb5591ae4889a054b63bcbd7fc98.
    
    crs* (int): geographic coordinate system as EPSG code. You can find the codes at https://spatialreference.org/ref/epsg/
    
    output_file (str): optional; full path to the shapefile wiht extension ".shp" where the output file will be 
    saved. Please add "r" before the string or use double slash "\\" instead of single slash in the path.
    
    Example:
    tnc = get_AGOL_layer(layer_id = 'de8681a325f643f49a1fc848a0dac5bb', 
                        crs = 3310,
                        output_file = r'C:\Documents\TNC_lands.shp')
    
    Please address your questions to Ira Koroleva (irina.koroleva@tnc.org)
    The Nature Conservancy, 2023
    """
    
    # log into AGOL
    link = 'https://www.arcgis.com/sharing/rest'
    gis = GIS(link, client_id='ztKl8vv9x4R8bE20')
    
    # Get the hosted feature layer
    flayer = gis.content.get(layer_id).layers[0]
    # .query() returns a FeatureSet
    fset = flayer.query()
    # Get a GeoJSON string representation of the FeatureSet
    gjson_string = fset.to_geojson
    # Read GeoJSON string into a dict
    gjson_dict = json.loads(gjson_string)
    # Read in as geodataframe
    layer = gpd.GeoDataFrame.from_features(gjson_dict['features'], crs=3857)
    
    # Project layer
    layer = layer.to_crs(crs)
    
    # Check validity
    total = len(layer['geometry'])
    valid = len(layer.is_valid)
    invalid = total - valid
    
    # Multipolygons might have been turned into polygons and their parts became invalid, fix them by turning them back into multipolygons
    if invalid > 0:
        layer['geometry'] = layer['geometry'].apply(make_valid)
        print(invalid + ' invalid features were fixed')
    
    # Save as shapefile if the location is provided
    if not (output_file is None):
        # Change any date fields to text
        layer_save = layer.copy()
        cols = list(layer_save)
        for i in cols:
            if layer_save[i].dtype == 'datetime64[ns]': 
                layer_save[i] = layer_save[i].astype(str)
        
        layer_save.to_file(output_file) 
    return layer

In [ ]:
# Example - loading TNC lands
tnc = get_AGOL_layer(layer_id = 'f77e630cfe914483929f2a0bfba230c3', 
                    crs = 4326,
                    output_file = r'C:\Users\jinsu.elhance\file.shp');

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://www.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=ztKl8vv9x4R8bE20&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=n9jc0095YcPgQielW0Ev0lxhW6QxgD&allow_verification=false


In [14]:
arcpy.SignInToPortal("https://tnc.maps.arcgis.com") ##Authentication
p_info = arcpy.GetPortalDescription("https://tnc.maps.arcgis.com")
print(f"Signed in as {p_info['user']['username']}")

Signed in as dangermond_preserve@tnc.org_TNC


In [15]:
#Pointer for portal data access
portal = GIS("https://tnc.maps.arcgis.com")

# Loading Portal Data Into Workbook

## Attach workbook to your ArcGIS Project

In [16]:
project_dir = "C:/Users/jinsu.elhance/Documents/ArcGIS/Projects/FinalRoadUse/"
aprx = arcpy.mp.ArcGISProject(f"{project_dir}/FinalRoadUse.aprx")

try:
    mp = aprx.listMaps()[0]
except IndexError:
    print("Please add a map to your project")
    
arcpy.env.workspace = f"{project_dir}/FinalRoadUse.gdb"

## Create or Identify Directories for GeoJSON and Shapefile Outputs

In [17]:
if not os.path.exists(f"{project_dir}/GeoJSONs"):
    os.makedirs(f"{project_dir}/GeoJSONs")
geojson_dir = f"{project_dir}/GeoJSONs"
    
if not os.path.exists(f"{project_dir}/Shapefiles"):
    os.makedirs(f"{project_dir}/Shapefiles")
shapefile_dir = f"{project_dir}/Shapefiles"

# Useful Functions

In [22]:
def getFields(layer):
    assert type(layer) == arcpy._mp.Layer
    _dsc = arcpy.da.Describe(layer)
    if _dsc.get('children', False):
        print("This layer is a grouped layer")
        return None
    return arcpy.da.Describe(layer)['fields']

def getFieldNames(layer):
    assert type(layer) == arcpy._mp.Layer
    _dsc = arcpy.da.Describe(layer)
    if _dsc.get('children', False):
        print("This layer is a grouped layer")
        return None
    return [field.name for field in arcpy.da.Describe(layer)['fields']]

def LayerToGDF(project, layer, crs):
    """
    Input: 
    project (arcpy._mp.ArcGISProject) : Current working project
    layer (arc._mp.Layer) : layer with data to be converted to a geodataframe
    crs (string) : EPSG crs that you want layer to be projected to. Defaults to 'EPSG:4326'
    Output:
    returns geopandas geodataframe from layer
    """
    assert type(project) == arcpy._mp.ArcGISProject and geojson_dir
    output_file = arcpy.da.Describe(layer)['aliasName']
    
    if (os.path.exists(f"{geojson_dir}/{output_file}.geojson")):
        _owrite = input(f"This GeoJSON ({output_file}) has already been created, would you like to overwite (y/n)").lower()
        if _owrite == "y":
            os.remove(f"{geojson_dir}/{output_file}.geojson")
            LayerToGDF(project, layer, crs)
        else:
            return gpd.GeoDataFrame.from_file(f"{geojson_dir}/{output_file}.geojson").to_crs(crs)

    arcpy.conversion.FeaturesToJSON(layer, f"{geojson_dir}/{output_file}.geojson", geoJSON = True)
    return gpd.GeoDataFrame.from_file(f"{geojson_dir}/{output_file}.geojson").to_crs(crs)

def GDFToLayer(gdf, out_features, geometry_type, project):
    """
    Inputs:
    gdf (GeoDataFrame)
    out_features (string)
    geometry_type (string)
    project (arcpy._mp.ArcGISProject)
    Output:
    None -> Prints status message
    """
    #Assert Geojson dir and shapefile dir are defined
    assert geojson_dir and shapefile_dir
    
    if (os.path.exists(f"{shapefile_dir}/{out_features}.shp")):
        _owrite = input(f"This Shapefile ({out_features}) has already been created, would you like to overwite (y/n)").lower()
        if _owrite == "y":
            os.remove(f"{geojson_dir}/{out_features}.geojson")
            for f in glob.glob(f"{shapefile_dir}/{out_features}.*"):
                os.remove(f)
            GDFToLayer(gdf, out_features, geometry_type, project)
        else: 
            arcpy.conversion.FeatureClassToGeodatabase(f"{shapefile_dir}/{out_features}.shp", project.defaultGeodatabase)
            return
        
    else: #Create new GeoJSON and shp
        #Write GeoDataFrame to a GeoJSON in the geojson_dir 
        gdf.to_file(f"{geojson_dir}/{out_features}.geojson", driver="GeoJSON")
        #Read new GeoJSON to a shapefile
        arcpy.conversion.JSONToFeatures(f"{geojson_dir}/{out_features}.geojson", f"{shapefile_dir}/{out_features}", geometry_type = geometry_type)
    
    #Add shapefile to project as a layer
    arcpy.conversion.FeatureClassToGeodatabase(f"{shapefile_dir}/{out_features}.shp", project.defaultGeodatabase)
    
    print("Successfully added to geodatabase")
    return

# Perform Analysis

### Pull Layers into GeoDataFrames

In [23]:
layer_list = [layer.name for layer in mp.listLayers()]
roads_jldp = mp.listLayers()[0]
lora_tracking_jldp = mp.listLayers()[1]

roads_jldp_gdf = LayerToGDF(aprx, roads_jldp, "EPSG:4326")
lora_jldp_gdf = LayerToGDF(aprx, lora_tracking_jldp, "EPSG:4326")

This GeoJSON (jldp_roads_root_2022_02) has already been created, would you like to overwite (y/n)n
This GeoJSON (lora_tracking_3) has already been created, would you like to overwite (y/n)n


### Clip Data to JLDP Boundary

In [24]:
jldp_bounds = gpd.read_file("C:/Users/jinsu.elhance/Box/000. Jinsu Elhance/HelpfulShapes/Shapefiles/DP_Boundary.shp")

lora_jldp_gdf_clipped = gpd.clip(lora_jldp_gdf, jldp_bounds)
roads_jldp_gdf_clipped = gpd.clip(roads_jldp_gdf, jldp_bounds)

### Generate Near Table

In [25]:
try:
    arcpy.analysis.GenerateNearTable(lora_tracking_jldp, roads_jldp, "lora_road_near_table", "20 meters", closest=True)
except:
    print("File already exists")

File already exists


In [26]:
near_table_ptr = arcpy.ListTables("lora_road_near_table")[0]
_fieldnames = [field.name for field in arcpy.da.Describe(near_table_ptr)['fields']]

### Joining Lora Tracking Data to Roads

In [27]:
near_table = gpd.GeoDataFrame(arcpy.da.TableToNumPyArray("lora_road_near_table", _fieldnames))

In [28]:
_lora_road_join = (lora_jldp_gdf_clipped
    .merge(near_table, left_on = "FID", right_on="IN_FID", how="left")
    .rename(columns={'NEAR_FID':'road_ID'}))

_lora_road_join['date'] = _lora_road_join['rec_tm_utc'].dt.floor("D")

daily_road_use = (_lora_road_join
                  .groupby(['road_ID', 'date', 'dev'])[['road_ID', 'date', 'dev']]
                  .agg(lambda x: None or x))

daily_road_use['month'] = _lora_road_join['date'].dt.strftime('%Y/%m')
daily_road_use['year'] = _lora_road_join['date'].dt.strftime('%Y')

monthly_road_use = (daily_road_use
                    .groupby(['road_ID', 'month'])[['dev']]
                    .count()
                    .rename(columns = {"dev":'vehicle_count'})
                    .reset_index())

yearly_road_use = (daily_road_use
                   .groupby(['road_ID', 'year'])[['dev']]
                   .count()
                   .rename(columns = {"dev":'vehicle_count'})
                   .reset_index())

total_road_use = (daily_road_use
                  .groupby(['road_ID'])[['dev']]
                  .count()
                  .rename(columns = {"dev": 'vehicle_count'})
                  .reset_index())

In [40]:
total_road_use_lines = gpd.GeoDataFrame(roads_jldp_gdf_clipped
                       .merge(total_road_use, left_on="FID", right_on="road_ID", how="left")
                       .set_index('road_ID'))

total_road_use_lines.head()

,FID,Type,Road_Compo,Condition,Safety_Lev,Notes,Length,name,road_type,vehicle_ty,...,comments,compositio,Tier,Flag,Pasture,Zone,GlobalID,Shape__Length,geometry,vehicle_count
road_ID,,,,,,,,,,,,,,,,,,,,,
252.0,252,Rugged - 2x2,Dirt/Grass,Poor,4.0,,290.599869,CMT Road,Primary Dirt Road,AWD Vehicle,...,,Dirt/Grass,Tier 2,Restoration,54,Cojo Coast,{43840fc8-7bbd-4187-8e8a-a60c28bd7c56},953.409737,"LINESTRING Z (-120.42018 34.45319 0.00000, -12...",236.0
251.0,251,Rugged - 2x2,Asphalt,Good,5.0,,174.457302,Cojo Bay Road,Paved Private Road,2WD Vehicle,...,,Asphalt,Tier 1,,54,Cojo Coast,{caeed10b-cbd1-4469-ac19-0c3993e111a7},284.358726,"LINESTRING Z (-120.42018 34.45319 0.00000, -12...",22.0
492.0,492,Rugged - 2x2,Asphalt,Good,5.0,,39.041557,Cojo Bay Road,Paved Private Road,2WD Vehicle,...,,Asphalt,Tier 1,,Steve's Flat?,Cojo Coast,{893e189c-830f-4e25-8e1e-150728b6505e},128.088841,MULTILINESTRING Z ((-120.42024 34.45353 0.0000...,3.0
259.0,259,Rugged - 4x4,Dirt/Grass,Poor,3.0,washouts,729.883927,UTV Test,Backcountry Road,ATV/UTV,...,,Dirt/Grass,Tier 3,Rain,Steve's Flat?,Cojo Coast,{ebd68f6a-4b6d-4919-a95f-ea557ca0b7bf},2394.627519,MULTILINESTRING Z ((-120.42024 34.45359 0.0000...,9.0
NaN,793,Rugged - 2x2,Asphalt,Good,5.0,,6.743531,Cojo Bay Road,Paved Private Road,2WD Vehicle,...,,Asphalt,Tier 1,,Steve's Flat?,Cojo Coast,{855ba6ce-6b25-4b38-a6b8-0d5078a1235e},22.124400,"LINESTRING Z (-120.42024 34.45359 0.00000, -12...",NaN


In [41]:
monthly = (roads_jldp_gdf_clipped
    .merge(monthly_road_use, left_on="FID", right_on="road_ID", how="right")[['road_ID', 'geometry', 'month', 'vehicle_count']]
    .sort_values('month')
    .set_index('road_ID'))

#monthly.head()

### Write Data Back to ArcGIS Project

In [42]:
GDFToLayer(monthly, "monthly_road_use", "POLYLINE", aprx)
GDFToLayer(total_road_use_lines, "total_road_use", "POLYLINE", aprx)

This Shapefile (monthly_road_use) has already been created, would you like to overwite (y/n)n
Successfully added to geodatabase


### Attempted Methods

In [22]:
lora_tracking_jldp_df = pd.DataFrame.spatial.from_featureclass("lora_tracking_3")